In [2]:
%iam_role arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 0.32 and you have 0.31 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Current iam_role is None
iam_role has been set to arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role.


In [4]:
%region us-west-2

Previous region: None
Setting new region to: us-west-2
Region is set to: us-west-2


In [0]:
%additional_python_modules "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl"
%number_of_workers 2
%connections dev-glue-redshift-connection
#%idle_timeout 30

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Additional python modules to be included:
s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl
Previous number of workers: 5
Setting new number of workers to: 2
Connections to be included:
dev-glue-redshift-connection
Authenticating with profile=default
glue_role_arn defined by user: arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 2
Session ID: 920746cc-7378-4c8b-8bce-c4db38a1e3b7
Applying the following default arguments:
--glue_kernel_version 0.31
--enable-glue-datacatalog true
--additional-python-modules s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl
Waiting for session 920746cc-7378-4c8b-8bce-c4db38a1e3b7 to get into ready status...
Session 920746cc-7378-4c8b-8bce-c4db38a1e3b7 has been created




In [11]:
#download configuration file
import boto3
session = boto3.Session()

s3 = session.resource('s3')
s3.Bucket('dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o').download_file('confs/filevine/sstm.yaml', 'sstm.yaml')

In [12]:
with open("sstm.yaml", "r") as f:
    sstm_config = f.read()
    

In [3]:
def readFromFile(ext, path) :
    fileContent = spark.read.format(ext).option("header", "true").option("mergeSchema", "true").load(path)
    return fileContent


In [4]:
def writeToFile(df_join , path, mode) :
    df_join.write.parquet(path,mode=mode)

In [5]:
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")


In [6]:
from sstm_transformation.tsm_builder import TSMBuilder

In [7]:
#Read All dfs 

df_project =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/project.parquet")
df_intake =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/intake.parquet")
df_contact =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/contact/")
df_projecttype = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/projecttype/")
df_meds = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/collections/meds.parquet")
df_casesummary =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/casesummary.parquet")


In [13]:
builder = TSMBuilder("sstm.yaml", spark=spark)


In [9]:
#Upload data on S3 
from awsglue.dynamicframe import DynamicFrame
def upload_result(result):
    temp_bucket = "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp"
    for key, df in result.items():
        print(f"Writing {key}")
        writeToFile(df,"s3://dev-truve-devops-05-databr-bucketetlprocesseddata-h2m2xopoctot/{}/".format(key.lower()), "overwrite")
        dyf = DynamicFrame.fromDF(dataframe=df, glue_ctx=glueContext, name=key)
        table_name = key.lower()
        dyf.show(10)
        #break
        redshift_connection_options = {"url": "jdbc:redshift://dev-redshift-cluster.cxorr0thbkvd.us-west-2.redshift.amazonaws.com:5439/truve", 
                        "user": "truveadmin", 
                        "password": "YuXpRNej45S1E7mT",
                        "preactions" : "delete from tsm.{}".format(table_name),
                        "dbtable": "tsm.{}".format(table_name), 
                        "redshiftTmpDir": temp_bucket} 
        datasink4 = glueContext.write_dynamic_frame.from_options(frame=dyf,
                                                        connection_type="redshift",
                                                        connection_options=redshift_connection_options)
        #break
    

In [28]:
#PeopleType Transformation
def people_type_transformation(df_contact):
    #Read Raw Data
    result = builder.build_peopletypes(contact_df=df_contact)
    #print(result)
    upload_result(result)
    
people_type_transformation(df_contact)

Writing CMS_PeopleType
{"People_Type": "", "Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Custom1": null, "Custom2": null, "Custom3": null, "People_Sub_Type": null, "People_Type_Id": 1}
{"People_Type": "Adjuster", "Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Custom1": null, "Custom2": null, "Custom3": null, "People_Sub_Type": null, "People_Type_Id": 2}
{"People_Type": "Attorney", "Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Custom1": null, "Custom2": null, "Custom3": null, "People_Sub_Type": null, "People_Type_Id": 3}
{"People_Type": "Client", "Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Custom1": null, "Custom2": null, "Custom3": null, "People_Sub_Type": null, "People_Type_Id": 4}
{"People_Type": "Court", "Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Custom1": null, "Custom2": null, "Custom3": null, "People_Sub_Type": null, "People_Type_Id": 5}
{"People_Type": "Defendant", "Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Custom1": null, "Custom2": null, "Custom3": null, "Pe

In [29]:
#Peoples Transformation
def people_master_transformation(df_contact):
    #Read Raw Data
    result = builder.build_peoplemaster(contact_df=df_contact)
    print(result)
    upload_result(result)
    
people_master_transformation(df_contact)

root
 |-- People_ID: string (nullable = true)
 |-- People_Type: string (nullable = true)

root
 |-- People_Type: string (nullable = true)
 |-- Truve_Org_ID: integer (nullable = true)
 |-- Client_Org_ID: string (nullable = true)
 |-- Custom1: string (nullable = true)
 |-- Custom2: string (nullable = true)
 |-- Custom3: string (nullable = true)
 |-- People_Sub_Type: string (nullable = true)
 |-- People_Type_Id: integer (nullable = true)

{'CMS_People': DataFrame[Truve_Org_ID: int, Client_Org_ID: string, People_ID: int, First_Name: string, Middle_Name: string, Last_Name: string, Date_of_Birth: date, Gender: string, Custom1: string, Custom2: string, Custom3: string, Team_ID: int], 'CMS_PeoplePeopleTypes': DataFrame[People_ID: int, People_Type_ID: int, Truve_Org_ID: int, Client_Org_ID: string]}
Writing CMS_People
{"Truve_Org_ID": 6586, "Client_Org_ID": "6586", "People_ID": 17882226, "First_Name": "Example", "Middle_Name": null, "Last_Name": "Insurer", "Date_of_Birth": null, "Gender": null, 

In [30]:
#Peoples Transformation
def projecttype_transformation(df_projecttype):
    #Read Raw Data
    result = builder.build_practicetypes(df_projecttype)
    print(result)
    upload_result(result)
    
projecttype_transformation(df_projecttype)

root
 |-- Truve_Org_ID: integer (nullable = false)
 |-- Client_Org_ID: string (nullable = false)
 |-- Practice_Type_ID: integer (nullable = true)
 |-- Practice_Type_Name: string (nullable = true)
 |-- Custom1: string (nullable = true)
 |-- Custom2: string (nullable = true)
 |-- Custom3: string (nullable = true)

{'CMS_PracticeTypes': DataFrame[Truve_Org_ID: int, Client_Org_ID: string, Practice_Type_ID: int, Practice_Type_Name: string, Custom1: string, Custom2: string, Custom3: string]}
Writing CMS_PracticeTypes
{"Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Practice_Type_ID": 18764, "Practice_Type_Name": "ALI AWAD LAW TEMPLATE", "Custom1": null, "Custom2": null, "Custom3": null}
{"Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Practice_Type_ID": 29908, "Practice_Type_Name": "Baby Formula / NEC cases", "Custom1": null, "Custom2": null, "Custom3": null}
{"Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Practice_Type_ID": 30531, "Practice_Type_Name": "CEO Lawyer Summit", "Custom1": null,

In [31]:
#Peoples Transformation
df_projecttype = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/projecttype/")

def phases_transformation(df_projecttype):
    #Read Raw Data
    
    result = builder.build_phasemaster(df_projecttype)
    print(result)
    upload_result(result)
    
phases_transformation(df_projecttype)

+------------+-------------+--------+--------------------+----------------+-----------+--------------+------------------+-------+-------+-------+
|Truve_Org_ID|Client_Org_ID|Phase_ID|          Phase_Name|Practice_Type_ID|Phase_Order|Phase_Category|Phase_Sub_Category|Custom1|Custom2|Custom3|
+------------+-------------+--------+--------------------+----------------+-----------+--------------+------------------+-------+-------+-------+
|        6586|         6586|   91388|            New Lead|           18764|       null|          null|              null|   null|   null|   null|
|        6586|         6586|  103016|              Intake|           18764|       null|          null|              null|   null|   null|   null|
|        6586|         6586|   91389|         File Set-Up|           18764|       null|          null|              null|   null|   null|   null|
|        6586|         6586|   91390|           Treatment|           18764|       null|          null|              null|   

In [32]:
#Peoples Transformation
def value_transformation(df_meds):
    #Read Raw Data
    result = builder.build_casefigures(df_meds)
    print(result)
    upload_result(result)
    
value_transformation(df_meds)

+------------+-------------+--------------+--------+--------------+--------------------+-----------+-------------+--------+-------+-------+-------+
|Truve_Org_ID|Client_Org_ID|Parent_Case_ID| Case_ID|Case_Figure_ID|         Figure_Type|Figure_Date|Figure_Status|   Value|Custom1|Custom2|Custom3|
+------------+-------------+--------------+--------+--------------+--------------------+-----------+-------------+--------+-------+-------+-------+
|        6586|         6586|       9675645| 9675645|          null|       Case Expenses|       null|         null|    20.0|   null|   null|   null|
|        6586|         6586|       9675645| 9675645|          null|       Case Expenses|       null|         null|   72.88|   null|   null|   null|
|        6586|         6586|       9675645| 9675645|          null|       Case Expenses|       null|         null|   72.88|   null|   null|   null|
|        6586|         6586|       9675645| 9675645|          null|       Case Expenses|       null|         nul

In [33]:
#Peoples Transformation
def cases_transformation(df_project):
    #Read Raw Data
    result = builder.build_casemaster(df_project)
    print(result)
    upload_result(result)
    
cases_transformation(df_project)

+--------------------+
|    Practice_Type_ID|
+--------------------+
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 25962,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 25962,...|
|{'native': 25962,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 25962,...|
+--------------------+
only showing top 20 rows

root
 |-- Truve_Org_ID: integer (nullable = false)
 |-- Client_Org_ID: string (nullable = false)
 |-- Case_ID: integer (nullable = true)
 |-- Practice_Type_ID: integer (nullable = true)
 |-- Is_Archived: boolean (nullable = true)
 |-- Incident_Date: date (nullable = true)

+------------+-------------+--------+----------------+-----------+-------------+
|Truve_Org_ID|Client_Org_ID| Case_ID|Practice_Type_ID|Is_Archived|I

In [34]:
#Peoples Transformation
def intake_transformation(df_intake):
    #Read Raw Data
    result = builder.build_intakesummary(df_intake)
    return result 

result = intake_transformation(df_intake)
#result["IntakeDetails"] = result["IntakeDetails"].drop("referral_fee")
upload_result(result)


+------------+-------------+--------------+--------+---------+---------------------------+-------------+--------------+----------------+----------------+------------+-------------------+-----------------+-----------+-------+-------+-------+
|Truve_Org_ID|Client_Org_ID|Parent_Case_ID| Case_ID|Intake_ID|Person_Performing_Intake_ID|Intake_Source|Date_of_Intake|Date_of_Incident|DUI_or_HitandRun|Referral_Fee|If_Case_Referred_In|If_Qualified_Case|If_VIP_Lead|Custom1|Custom2|Custom3|
+------------+-------------+--------------+--------+---------+---------------------------+-------------+--------------+----------------+----------------+------------+-------------------+-----------------+-----------+-------+-------+-------+
|        6586|         6586|       9252629| 9252629|  9252629|                   30625608|         null|    2021-04-28|      2021-04-22|            None|        null|               null|             null|       null|   null|   null|   null|
|        6586|         6586|       9

In [35]:
#Peoples Transformation
def casesummary_transformation(df_casesummary):
    #Read Raw Data
    result = builder.build_casesummary(df_casesummary)
    print(result)
    upload_result(result)
    
casesummary_transformation(df_casesummary)

+------------+-------------+--------------+--------+------------+----------------+----------------+---------+-------------------+-----------+--------------------------+-----------------------+--------------+------------+------------+------------+--------------+---------------------+----------------+-----------------------+-------------------------+---------------------------+----------------------+------------------------+-----------------------+-----------+--------------------+-------------+-------+-------+-------+-------+-------+
|Truve_Org_ID|Client_Org_ID|Parent_Case_ID| Case_ID|Case_Type_ID|Case_Create_Date|Date_of_Incident|Case_Name|Plaintiff_Full_Name|Attorney_ID|Prelitigation_Paralegal_ID|Litigation_Paralegal_ID|CaseManager_ID|Cocounsel_ID|Case_Team_ID|Insurance_ID|Case_Status_ID|Case_Marketing_Source|Case_Source_Name|Attorney_Fee_Percentage|Projected_Settlement_Date|Projected_Settlement_Amount|Actual_Settlement_Date|Actual_Settlement_Amount|If_Case_Settled_Presuit|If_VIP_Case|

In [ ]:
builder._get_table_config("Phases")
sstm_config

In [ ]:
def people_master_transformation():
    #Read Raw Data
    contact_dyf = glueContext.create_dynamic_frame_from_options(\
    connection_type = "s3", \
    connection_options = {
        "paths": ["s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/contact/historical_contacts.parquet"]}, \
    format = "parquet",
    )
    contact_sp_df = contact_dyf.toDF()
    print(contact_sp_df.count())
    sstm_peoplemaster_df = builder.build_peoplemaster(contact_df=contact_sp_df)
    trans_obj = DynamicFrame.fromDF(sstm_peoplemaster_df, glueContext, "sstm_peoplemaster")
    print(trans_obj)
    #sstm_peoplemaster_df.write.mode("overwrite").format("parquet").save("s3://dev-truve-devops-05-databr-bucketetlprocesseddata-h2m2xopoctot/peoplemaster/")
    #print("PeopleTypes Processed successfully")

In [ ]:
#people_type_transformation()
people_master_transformation()

In [ ]:
df_forms =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/casesummary.parquet")


In [ ]:
df_forms.count()

In [ ]:
df_forms.printSchema()

In [ ]:
df_forms.show()

In [ ]:
from awsglue.dynamicframe import DynamicFrame

dyf_casesummary = DynamicFrame.fromDF(df_forms, glueContext, "nested")


In [ ]:
glueContext.write_dynamic_frame.from_options(frame = dyf_casesummary,
                                         connection_type = "s3",
                                         connection_options = {"path": "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/casesummary/"},
                                         format = "parquet")

In [ ]:
writeToFile(df_forms,"s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/casesummary/", "overwrite")


In [9]:
%stop_session

Stopping session: 920746cc-7378-4c8b-8bce-c4db38a1e3b7
Stopped session.


In [ ]:
df_forms =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/collection/meds.parquet")


In [ ]:
df_forms.count()

In [ ]:
df_forms.printSchema()

In [ ]:
df_forms.show(n=10)

In [ ]:
writeToFile(df_forms,"s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/meds/", "overwrite")


In [ ]:
df_intake =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/intake.parquet")


In [ ]:
df_intake.count()

In [ ]:
df_intake.show(n=10)

In [ ]:
df_intake.printSchema()

In [ ]:
writeToFile(df_intake,"s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/intake/", "overwrite")


In [ ]:
df_casesummary =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/casesummary.parquet")


In [ ]:
df_casesummary.count()

In [ ]:
df_casesummary.show()

In [ ]:
def write_parquet_files(df, name):
    df_rept = df.repartition(1)
    writeToFile(df_rept,"s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/{}/".format(name), "overwrite")


In [ ]:
df_intake =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/intake.parquet")


In [ ]:
write_parquet_files(df_intake, "intake")

In [ ]:
df_intake.count()

In [ ]:
df_project =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/project.parquet")
write_parquet_files(df_project, "project")

In [ ]:
df_meds =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/collections/meds.parquet")
write_parquet_files(df_meds, "meds")

In [ ]:
result = builder.build_peopletypes(contact_df=df_contact)

In [ ]:
result

In [ ]:
people_type_df = result["PeopleTypes"]

In [ ]:
people_type_df.printSchema()

In [ ]:
from pyspark.sql import Window 
from pyspark.sql import functions as F

window = Window.orderBy(F.col('People_Type'))
rs_people_type_df = people_type_df.withColumn("People_Type_Id", F.row_number().over(window))

In [ ]:
from pyspark.sql.types import *
rs_people_type_df = rs_people_type_df.withColumn("Client_Org_ID", rs_people_type_df["Client_Org_ID"].cast(StringType()))

In [ ]:
rs_people_type_df = rs_people_type_df.withColumn("People_Type_Id", rs_people_type_df["People_Type_Id"].cast(IntegerType()))

In [ ]:
rs_people_type_df.printSchema()

In [ ]:
dyf_rs_people_type = DynamicFrame.fromDF(dataframe=rs_people_type_df, glue_ctx=glueContext, name="shiv")
#dyf = DynamicFrame.fromDF(dataframe=df, glue_ctx=glueContext, name=key)
        

In [ ]:
temp_bucket = "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp"
    
table_name = "cms_peopletype"
redshift_connection_options = {"url": "jdbc:redshift://dev-redshift-cluster.cxorr0thbkvd.us-west-2.redshift.amazonaws.com:5439/truve", 
                        "user": "truveadmin", 
                        "password": "YuXpRNej45S1E7mT",
                        "preactions" : "delete from tsm.{}".format(table_name),
                        "dbtable": "tsm.{}".format(table_name), 
                        "redshiftTmpDir": temp_bucket} 
        
datasink4 = glueContext.write_dynamic_frame.from_options(frame=dyf_rs_people_type,
                            connection_type="redshift",
                            connection_options=redshift_connection_options)
        

In [ ]:
result = builder.build_peoplemaster(contact_df=df_contact)

In [ ]:
people_df = result["Peoples"]

In [ ]:
people_df = people_df.withColumn("Client_Org_ID", people_df["Client_Org_ID"].cast(StringType()))

In [ ]:
people_df = people_df.withColumn("People_ID", people_df["People_ID"].cast(IntegerType()))
people_df = people_df.withColumn("Date_of_Birth", people_df["Date_of_Birth"].cast(DateType()))
people_df = people_df.withColumn("Team_ID", F.lit(1).cast(IntegerType()))


In [ ]:
people_df.printSchema()

In [ ]:
people_df.show(n=10)

In [ ]:
dyf_people_type = DynamicFrame.fromDF(dataframe=people_df, glue_ctx=glueContext, name="shiv")


        

In [ ]:
table_name = "cms_people"
redshift_connection_options = {"url": "jdbc:redshift://dev-redshift-cluster.cxorr0thbkvd.us-west-2.redshift.amazonaws.com:5439/truve", 
                        "user": "truveadmin", 
                        "password": "YuXpRNej45S1E7mT",
                        "preactions" : "delete from tsm.{}".format(table_name),
                        "dbtable": "tsm.{}".format(table_name), 
                        "redshiftTmpDir": temp_bucket} 
        
datasink4 = glueContext.write_dynamic_frame.from_options(frame=dyf_people_type,
                            connection_type="redshift",
                            connection_options=redshift_connection_options)

In [ ]:
contact_type_df = result["Peoples_PeopleType"]

In [ ]:
contact_type_df = contact_type_df.withColumn("Truve_Org_ID", F.lit(6586).cast(IntegerType()))
contact_type_df = contact_type_df.withColumn("Client_Org_ID", F.lit("6586").cast(StringType()))
contact_type_df = contact_type_df.withColumn("People_ID", contact_type_df["People_ID"].cast(IntegerType()))
contact_type_df = contact_type_df.withColumn("People_Type_ID", contact_type_df["People_Type_ID"].cast(IntegerType()))
        


In [ ]:
contact_type_df.print

In [ ]:
dyf_contact_type = DynamicFrame.fromDF(dataframe=contact_type_df, glue_ctx=glueContext, name="shiv")


In [ ]:
table_name = "cms_peoplepeopletypes"
redshift_connection_options = {"url": "jdbc:redshift://dev-redshift-cluster.cxorr0thbkvd.us-west-2.redshift.amazonaws.com:5439/truve", 
                        "user": "truveadmin", 
                        "password": "YuXpRNej45S1E7mT",
                        "preactions" : "delete from tsm.{}".format(table_name),
                        "dbtable": "tsm.{}".format(table_name), 
                        "redshiftTmpDir": temp_bucket} 
        
datasink4 = glueContext.write_dynamic_frame.from_options(frame=dyf_contact_type,
                            connection_type="redshift",
                            connection_options=redshift_connection_options)

In [ ]:
result = builder.build_practicetypes(df_projecttype)

In [ ]:
practice_type_df = result["PracticeTypes"]

In [ ]:
practice_type_df.printSchema()

In [ ]:
practice_type_df = practice_type_df.withColumn("Client_Org_ID", practice_type_df["Client_Org_ID"].cast(StringType()))

In [ ]:
practice_type_df = practice_type_df.withColumn("Practice_Type_ID", practice_type_df["Practice_Type_ID"].cast(IntegerType()))

In [ ]:
dyf_practice_type = DynamicFrame.fromDF(dataframe=practice_type_df, glue_ctx=glueContext, name="shiv")


In [ ]:
table_name = "cms_practicetypes"
redshift_connection_options = {"url": "jdbc:redshift://dev-redshift-cluster.cxorr0thbkvd.us-west-2.redshift.amazonaws.com:5439/truve", 
                        "user": "truveadmin", 
                        "password": "YuXpRNej45S1E7mT",
                        "preactions" : "delete from tsm.{}".format(table_name),
                        "dbtable": "tsm.{}".format(table_name), 
                        "redshiftTmpDir": temp_bucket} 
        
datasink4 = glueContext.write_dynamic_frame.from_options(frame=dyf_practice_type,
                            connection_type="redshift",
                            connection_options=redshift_connection_options)

In [ ]:
%list_sessions

In [ ]:
%delete_session "07e6788a-3fd5-4358-a0cc-43b55c29e874"

In [ ]:
df = readFromFile("parquet", "s3://dev-truve-devops-05-databr-bucketetlprocesseddata-h2m2xopoctot/temp/casesummary.parquet")

In [ ]:
df.printSchema()

In [ ]:
from awsglue.dynamicframe import DynamicFrame
temp_bucket = "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp"

dyf = DynamicFrame.fromDF(dataframe=df, glue_ctx=glueContext, name="sh")
table_name = "cms_casedetails"
dyf.show(10)
#break
redshift_connection_options = {"url": "jdbc:redshift://dev-redshift-cluster.cxorr0thbkvd.us-west-2.redshift.amazonaws.com:5439/truve", 
                "user": "truveadmin", 
                "password": "YuXpRNej45S1E7mT",
                "preactions" : "delete from tsm.{}".format(table_name),
                "dbtable": "tsm.{}".format(table_name), 
                "redshiftTmpDir": temp_bucket} 
datasink4 = glueContext.write_dynamic_frame.from_options(frame=dyf,
                                                connection_type="redshift",
                                                connection_options=redshift_connection_options)